# **Sentiment Analysis with Deep Learning using BERT**




BERT is a large-scale transformer-based Language Model that can be finetuned for a variety of tasks.



In [1]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)



import os
for dirname, _, filenames in os.walk("/content/smile-annotations-final.csv"):
    for filename in filenames:
        print(os.path.join(dirname, filename))


## 1: Exploratory Data Analysis and Preprocessing

In [2]:
import torch
from tqdm.notebook import tqdm

In [3]:
df = pd.read_csv('smile-annotations-final.csv',
                names=['id', 'text', 'category'])
df.set_index('id', inplace=True)

In [4]:
df.head()

,text,category
id,,
611857364396965889,@aandraous @britishmuseum @AndrewsAntonio Merc...,nocode
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy
614877582664835073,@Sofabsports thank you for following me back. ...,happy
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy


In [5]:
df.category.value_counts()

,count
category,
nocode,1572
happy,1137
not-relevant,214
angry,57
surprise,35
sad,32
happy|surprise,11
happy|sad,9
disgust|angry,7


In [6]:
df = df[df.category.isin(['happy', 'not-relevant', 'angry', 'surprise', 'sad', 'disgust'])]

In [7]:
df.category.value_counts()

,count
category,
happy,1137
not-relevant,214
angry,57
surprise,35
sad,32
disgust,6


In [8]:
possible_labels = df.category.unique()

In [9]:
label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index

In [10]:
label_dict

{'happy': 0,
 'not-relevant': 1,
 'angry': 2,
 'disgust': 3,
 'sad': 4,
 'surprise': 5}

In [11]:
df.category = df['category'].map(label_dict)

In [12]:
df.head(10)

,text,category
id,,
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,0
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,0
614877582664835073,@Sofabsports thank you for following me back. ...,0
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,0
611570404268883969,@NationalGallery @ThePoldarkian I have always ...,0
614499696015503361,Lucky @FitzMuseum_UK! Good luck @MirandaStearn...,0
613601881441570816,Yr 9 art students are off to the @britishmuseu...,0
613696526297210880,@RAMMuseum Please vote for us as @sainsbury #s...,1
610746718641102848,#AskTheGallery Have you got plans to privatise...,1


Classes are imbalanced as visible

## 2: Training/Validation Split

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
X_train, X_val, y_train, y_val = train_test_split(df.index.values,
                                                  df.category.values,
                                                  test_size=0.15,
                                                  random_state=42,
                                                  stratify=df.category.values)

In [15]:
df['data_type'] = ['not_set']*df.shape[0]

In [16]:
df.head()

,text,category,data_type
id,,,
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,0,not_set
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,0,not_set
614877582664835073,@Sofabsports thank you for following me back. ...,0,not_set
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,0,not_set
611570404268883969,@NationalGallery @ThePoldarkian I have always ...,0,not_set


In [17]:
df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

In [18]:
df.groupby(['category', 'data_type']).count()

text
category data_type      
0        train       966
         val         171
1        train       182
         val          32
2        train        48
         val           9
3        train         5
         val           1
4        train        27
         val           5
5        train        30
         val           5

# 3. Loading Tokenizer and Encoding our Data

In [19]:
!pip install transformers

In [20]:
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

In [21]:
tokenizer = BertTokenizer.from_pretrained(
    'bert-base-uncased',
    do_lower_case=True
)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [22]:
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].text.values,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].text.values,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)

input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].category.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].category.values)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2760: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [23]:
dataset_train = TensorDataset(input_ids_train,
                              attention_masks_train,
                              labels_train)

dataset_val = TensorDataset(input_ids_val,
                            attention_masks_val,
                           labels_val)

In [24]:
len(dataset_train)

1258

In [25]:
dataset_val.tensors

(tensor([[  101,  1030,  8223,  ...,     0,     0,     0],
         [  101, 13970, 12269,  ...,     0,     0,     0],
         [  101,  1030,  2358,  ...,     0,     0,     0],
         ...,
         [  101,  2023,  2003,  ...,     0,     0,     0],
         [  101,  1030,  2329,  ...,     0,     0,     0],
         [  101,  2204,  2000,  ...,     0,     0,     0]]),
 tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]),
 tensor([1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 2, 0, 0, 0, 0, 0, 0, 2, 4, 0, 1, 0, 1, 0,
         0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 2, 4, 0, 0,
         1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 1, 0, 0, 0, 0, 1, 0, 4, 0, 2, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 2, 1, 1, 0, 3, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,

# 4. Setting up BERT Pretrained Model

In [26]:
from transformers import BertForSequenceClassification

In [27]:
model = BertForSequenceClassification.from_pretrained(
                                      'bert-base-uncased',
                                      num_labels = len(label_dict),
                                      output_attentions = False,
                                      output_hidden_states = False
                                     )

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# 5. Creating Data Loaders

In [28]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [29]:
batch_size = 4

dataloader_train = DataLoader(
    dataset_train,
    sampler=RandomSampler(dataset_train),
    batch_size=batch_size
)

dataloader_val = DataLoader(
    dataset_val,
    sampler=RandomSampler(dataset_val),
    batch_size=32
)

# 6. Setting Up Optimizer and Scheduler

In [30]:
from transformers import AdamW, get_linear_schedule_with_warmup

In [31]:
optimizer = AdamW(
    model.parameters(),
    lr = 1e-5,
    eps = 1e-8
)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [32]:
epochs = 10

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps = len(dataloader_train)*epochs
)

# 7. Defining our Performance Metrics

In [33]:
import numpy as np
from sklearn.metrics import f1_score

In [34]:
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average = 'weighted')

In [35]:
def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}

    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy:{len(y_preds[y_preds==label])}/{len(y_true)}\n')

# 8. Creating our Training Loop

In [36]:
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [37]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
print(device)

cuda


In [38]:
def evaluate(dataloader_val):

    model.eval()

    loss_val_total = 0
    predictions, true_vals = [], []

    for batch in tqdm(dataloader_val):

        batch = tuple(b.to(device) for b in batch)

        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():
            outputs = model(**inputs)

        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)

    loss_val_avg = loss_val_total/len(dataloader_val)

    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)

    return loss_val_avg, predictions, true_vals

In [39]:
for epoch in tqdm(range(1, epochs+1)):
    model.train()
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train,
                        desc='Epoch {:1d}'.format(epoch),
                        leave=False,
                        disable=False)

    for batch in progress_bar:
        model.zero_grad()
        batch = tuple(b.to(device) for b in batch)
        inputs = {
            'input_ids': batch[0],
            'attention_mask': batch[1],
            'labels': batch[2]
        }

        outputs = model(**inputs)
        loss = outputs[0]
        loss_train_total +=loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()

        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})

    #torch.save(model.state_dict(), f'Models/BERT_ft_Epoch{epoch}.model')

    tqdm.write('\nEpoch {epoch}')

    loss_train_avg = loss_train_total/len(dataloader_train)
    tqdm.write(f'Training loss: {loss_train_avg}')

    val_loss, predictions, true_vals = evaluate(dataloader_val)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (weighted): {val_f1}')


  0%|          | 0/10 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/315 [00:00<?, ?it/s]


Epoch {epoch}
Training loss: 0.6956511250090978


  0%|          | 0/7 [00:00<?, ?it/s]

Validation loss: 0.5863236359187535
F1 Score (weighted): 0.7948265964977566


...

Validation loss: 0.5427290541785104
F1 Score (weighted): 0.9056167968072378


Epoch 10:   0%|          | 0/315 [00:00<?, ?it/s]


Epoch {epoch}
Training loss: 0.028938061248562817


  0%|          | 0/7 [00:00<?, ?it/s]

Validation loss: 0.5475307809455054
F1 Score (weighted): 0.9043094841081615


# 9. Evaluating our Model

In [40]:
accuracy_per_class(predictions, true_vals)

Class: happy
Accuracy:164/171

Class: not-relevant
Accuracy:26/32

Class: angry
Accuracy:5/9

Class: disgust
Accuracy:0/1

Class: sad
Accuracy:2/5

Class: surprise
Accuracy:5/5



## saving model

In [41]:
output_dir = r"C:\Users\DELL\OneDrive\Desktop"


# Save the trained model to the specified directory
model.save_pretrained(output_dir)

# Save the tokenizer to the same directory
tokenizer.save_pretrained(output_dir)

('C:\\Users\\DELL\\OneDrive\\Desktop/tokenizer_config.json',
 'C:\\Users\\DELL\\OneDrive\\Desktop/special_tokens_map.json',
 'C:\\Users\\DELL\\OneDrive\\Desktop/vocab.txt',
 'C:\\Users\\DELL\\OneDrive\\Desktop/added_tokens.json')

In [42]:
!pip install -q gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 102.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 115.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 9.3 MB/s eta 0:00:00


In [43]:
#weights and bias file
torch.save({
    'model_state_dict': model.state_dict(),
}, "/content/fine_tuned_model.pth")
#from google.colab import files #downloading weights
#files.download("/content/fine_tuned_model.pth")


In [44]:
!pip install --upgrade gradio


In [45]:
!pip install streamlit torch transformers


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [53]:
!pip install gradio transformers torch



In [55]:
import gradio as gr
import torch
from transformers import BertTokenizer, BertForSequenceClassification

# Load the trained model and tokenizer
model_path =  r"C:\Users\DELL\OneDrive\Desktop" # Update this path if the model is stored on Google Drive
model = BertForSequenceClassification.from_pretrained(model_path)
tokenizer = BertTokenizer.from_pretrained(model_path)

# Define class names
class_names = ['happy', 'not-relevant', 'angry', 'surprise', 'sad', 'disgust']

# Define the sentiment analysis function
def analyze_sentiment(text):
    input_ids = tokenizer.encode(text, add_special_tokens=True)
    input_ids = torch.tensor(input_ids).unsqueeze(0)  # Add batch dimension
    with torch.no_grad():
        outputs = model(input_ids)
    predicted_class = torch.argmax(outputs.logits).item()
    sentiment = class_names[predicted_class]
    return sentiment

# Define the Gradio interface
def gradio_app(text):
    sentiment = analyze_sentiment(text)
    return f"**Sentiment:** {sentiment}"

iface = gr.Interface(
    fn=gradio_app,
    inputs=gr.Textbox(placeholder="Type your sentence here..."),
    outputs="text",
    title="Sentiment Analysis with BERT",
    description="Enter a sentence to analyze its sentiment."
)

# Launch the Gradio app
iface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://700629ecd2224e2266.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
